MLP: Multi layer perceptron  

**복습**
- cost function
- objective function (eg. MSE, Cross Entropy)
- backprop하려면 각 layer의 activation 값을 저장하고 있어야 함
- gradient descent

# 오늘할 내용
1. Activatoin Function
2. Batch Normalization
3. Optimization Method
4. Ensemble and Regularization: Overfitting 문제 해결

# Activation Functions

## non-linear을  사용해야 하는 이유?

- linear만을 쌓게 되면 단순히 모방하는 것에 지나지 않아서 의미가 없다. 따라서 non-linear을 사용해서 좀 더 complex하게 만들어야 한다.
- 하나의 직선(=linear)만을 사용해서 풀 수 없는 문제가 존재한다. 따라서 직선 하나에만 의존하지 않는 function을 만들어 complex한 decision boundary를 만들어야 한다.

## Non-linear function 종류

### sigmoid(x)

- $\sigma(x) =  1 / (1 + e^{-x})$
- 매우 작은 값과 매우 큰 값은 gradient가 0이 되서 backprop이 의미가 없어진다.
- layer를 깊게 쌓는 경우 sigmoid 사용하지 않는다.
- 하단에 있는 layer일 수록 backprop이 잘 되지 않는다.
- sigmoid를 과거에 많이 썼던 이유는 새로운 gradient를 계산하는 함수를 찾지 않고도 sigmoid 자체를 갖고 찾을 수 있기 때문에
- saturate: gradient가 사라진 상태 (평평한 부분)
- exp()는 계산비용이 큼
- sigmoid의 output의 평균이 0이 안된다는 문제가 있다. 이러면 업데이트할 수 있는 방향이 1,3 사분면만 해당되므로 2, 4 사분면에 최적값이 있다면 업데이트를 지그재그로 가야만 한다. 이는 비효율적이다.

### tanh(x)

- zero centered를 목적으로 만들어졌다.
- sigmoid와 tanh는 서로 형제 관계이다. 서로 표현 가능하다.
- $tanh(x) = 2 \times sigmoid(x) - 1$
- saturate는 해결하지 못한다.
- gradient vanishing: gradient가 0이 되어(kill) 앞 layer로 전달이 안 됐을 때
- gradient가 sigmoid보다 커서 학습이 빠르다.

### ReLU (Rectified Linear Unit)

- $f(x) = max(0,x)$
- gradient값이 떨어지지 않아서 gradient vanishing 문제가 덜 하다. 최근 딥러닝에서 점점 깊은 모델을 세우는데 앞의 activation보다 적합하다.
- 계산 비용이 매우 효율적이다. (값이 있다면 gradient는 1이다.)
- converge가 6배 정도 빠르다. 값이 줄어드는 경향이 sigmoid나 tanh보다 덜 해서
- zero-centered가 아니라는 문제, 지그재그 발생

#### ReLU로 복잡한 함수를 모방하려면?
여러 ReLU를 조합해서 원하는 function을 모방한다. 사용하는 activation에 따라서 만들어지는 decision boundary의 모양이 달라진다. ReLU는 직각모양이다.

### ReLU의 단점 (Dead ReLU)

- 해당되는 노드가 항상 0이라 gradient도 0이 되어 activation에 영향을 줄 수 없는 것을 의미한다.
- 어쩔 수 없이 나타날 수 밖에 없다.
- 해결하기 위해서 bias term을 준다.(eg. 0.01) 조금이라도 activation에 반영을 할 수 있도록

그래도 ReLU를 사용하는 이유는 weight initialize를 적절히 해주면 해결할 수도 있고, 죽는 애들보다 살아남는 애들이 더 많기 때문이다.

## Data Preprocessing

- 각각의 feature가 독립적이라는 가정 하에 mean과 std를 이용해서 normalization을 해준다.
- PCA(주성분 분석): feature의 가장 변동성이 높은 축으로 표현, 딥러닝에서는 많이 사용하지 않는다.


## gradient descent vs stochastic

- stochastic: 확률적으로
- batch gradient: 전체 데이터
- mini-batch gradient: batch를 쪼개서 gradient descent를 해주는 것, computational power 때문에 사용하긴 하지만 local minima를 탈출할 수 도 있다.

## Gradient Descent Update

x -= learning_rate * dx  
dx: 해당 점에서 gradient

## Momentum

v = mu * v - laerning_rate * dx  
x += v  
mu * v가 모멘텀에 해당하는 값이다. 이를 통해 이전 값을 조금씩 반영할 수 있다.

## Adagrad

- 이전까지의 그레이언트를 저장하고 있다가 변동성이 심하면 줄여주고 변동성이 적으면 키워줌
- 각 노드마다의 learning_rate를 다르게 해준 것과 비슷한 효과
- cache: 이전 그레디언트 값의 제곱 합
- cache += dx\*\*2
- x -= learning_rate * dx / (np.sqrt(cache) + 1e-7)
- iteration 증가하면 cache는 계속 증가하기만 하므로 나누는 분모가 계속 커져서 움직이지 않음.
- 해결책: 최근 것들만 따로 저장 = RMSProp

## RMSProp

- cache += decay_
- 모멘텀을 고려하지 않음

## Adam

- Momentum + RMSProp
- first moment: 과거 그레디언트 값 전체 (Momentum)
- second moment: 최근 그레디언트 값의 변동이 큰지 작은지 (AdaGrad / RMSProp)

### gradient descent 추가 설명(참고만 몰라도 됨)

- taylor expansion에서 나온 것
- 기울기가 점의 근방에서는 같지만 멀어질 수록 오차가 커짐
- 우리가 하는 gradient는 1차
- learning rate가 너무 크면 loss가 오히려 증가할 수 있음

팁: 성능이 안 오를 땐 learning_rate를 낮춰주거나 optimizer를 변경

## Learning Rate

learning rate schedule: 처음에는 learning rate를 조금 더 크게 주고 점점 learning rate를 줄여준다. 실제로도 많이 사용한다.

## Regularization

모델이 모두 외워버릴 수가 있다. 그걸 방지하는 것이 regularization

### Weight Regularization

- L1
- L2
- Elastic net

### 참고

training accuracy, training loss만으로 모델의 성능을 나타낼 수 없다. 70% 학습, 20% valid, 10% test.

### Drop-out

- 어떤 feature이나 node 없이 학습하게 되는 것
- 이렇게 되면 하나의 feature에 의존하지 않고 모든 feature가 의미있게
- 앙상블의 효과도 있다.
- drop out할 경우에는 test 할 때 scale up을 해준다. training할 때 본 데이터가 test할 때보다 적기 때문에
- test할 땐 drop out을 꺼야한다.

### Ensemble

여러 모델을 따로 학습시킨 다음에 여러 모델을 합쳐서 voting해서 일반화하려는 기법

## Batch Normalization

많은 모델의 성능을 향상시켰다. 이미지 분류에서 강력하게 사용된다. 인풋 레이어는 이미 normalization을 해줘서 문제가 없지만 layer를 거치면서 분포가 깨질 수도 있다. 그래서 layer을 거칠 때마다 분포를 정규분포화시켜준다. 이렇게 되면 gradient 값이 이상하게 구해지는 문제를 해결해준다. 추가로 regularization 효과도 줄 수 있다. 예를 들어, 우연히 batch에 안경을 쓴 데이터만 들어갈 경우 batch normlaization을 통해 안경에 대한 의존성을 줄여줄 수 있다. training data에 있는 mean과 std를 이용해서 batch norm을 해준다. test할 때도 마찬가지로 한다.

## 추천

relu, adam, learning_rate scheduler을 처음에 사용해 본다.

### normalization 종류
- batch
- layer
- instance
- group

cb_park@korea.ac.kr

# CNN

feature extraction을 하는 용도. cnn이 잘 작동하는 이유가 feature을 잘 뽑아내기 때문이다. 
staring 기법, receptive field  
conv는 local을 기준으로 본다. 따라서 회전을 해도 일치하는 패턴을 찾을 수 있어서 robust하다.

## 1. sliding window

filter에 있는 weight와 element-wise로 곱해준다. 모두 더한다. 더해준 갯수만큼 나눠주는 것은 옵션이다. 패턴이 일치할 수록 높은 값을 리턴한다. 또 다른 필터를 사용해서 반복적으로 계산한다. 이것을 conv 연산이라고 한다.

## 2. Pooling

- max pooling: 2\*2에서 가장 큰 값을 뽑아온다.
- classification할 때 중요한 정보만 남기는 것이다.
- width, height 기준으로 반토막한다.

## 3. Non-linear

### ReLU

- activation 값을 계산한다.
- 음수는 0으로 양수면 그대로

1,2,3을 반복한다. 마지막에 남은 output을 fully connect로 펴준다.

실제 데이터는 depth를 고려해야 하는데 자동으로 맞춰준다.